# A full example of how to use the package

## dataset

In [ ]:
# we will use DDI data set at https://github.com/isegura/DDICorpus
training_root = "./ddi/DDICorpusBrat/Train/MedLine"
testing_root = "./ddi/DDICorpusBrat/Test/MedLine"

## preprocessing using NLPpreprocessing package

> if you do not want to use this package, you can see the tutorial brat2bio.ipynb or https://github.com/nlplab/brat/blob/master/tools/anntoconll.py

In [ ]:
# download preprocessing package NLP
! git clone https://github.com/uf-hobi-informatics-lab/NLPreprocessing.git

In [ ]:
# link pacakge to python path
# import necessary functions
import sys
sys.path.append("./NLPreprocessing")
sys.path.append("./NLPreprocessing/text_process")
sys.path.append("./NLPpreprocessing/text_process/sentence_tokenization.py")

import logging
from annotation2BIO import generate_BIO, pre_processing, read_annotation_brat, BIOdata_to_file, logger
from sentence_tokenization import logger as logger1

# change log level to error to avoid too much log information in jupyter notebook
logger1.setLevel(logging.ERROR)
logger.setLevel(logging.ERROR)

In [ ]:
file_ids = set()

for fn in Path(training_root).glob("*.ann"):
    file_ids.add(fn.stem)
    
len(file_ids)

In [ ]:
# generate BIO from brat annotation
train_root = Path(training_root)
# train_bio = "./2018n2c2/bio/trains"
train_bio = "./ddi/ddi_bio/"
output_root = Path(train_bio)
output_root.mkdir(parents=True, exist_ok=True)

for fid in file_ids:
    txt_fn = train_root / (fid + ".txt")
    ann_fn = train_root / (fid + ".ann")
    bio_fn = output_root / (fid + ".bio.txt")
    
    txt, sents = pre_processing(txt_fn)
    e2idx, entities, rels = read_annotation_brat(ann_fn)
    nsents, sent_bound = generate_BIO(sents, entities, file_id=fid, no_overlap=False)
    
    BIOdata_to_file(bio_fn, nsents)

In [ ]:
# now we have to split the train and dev sets
# for transformer NER, we need to name these two datasets as train.txt and dev.txt
from sklearn.model_selection import train_test_split

file_ids = list(file_ids)
train_ids, dev_ids = train_test_split(file_ids, train_size=0.9, random_state=13, shuffle=True)
len(train_ids), len(dev_ids)

In [ ]:
import fileinput

merged = output_root / "merge" # this will the final data dir we use for training
merged.mkdir(exist_ok=True, parents=True)

# train
with open(merged / "train.txt", "w") as f:
    for fid in train_ids:
        f.writelines(fileinput.input(output_root / (fid + ".bio.txt")))
    fileinput.close()
        
# dev
with open(merged /"dev.txt", "w") as f:
    for fid in dev_ids:
        f.writelines(fileinput.input(output_root / (fid + ".bio.txt")))
    fileinput.close()

## training model

In [ ]:
"""
Next we will train the NER model

We will just use a BERT model pre-trained on general English corpora as an example

In general we need GPU to train the model, running with CPU the training will be extremely slow. 

To use GPU, you just need to run 'export CUDA_VISIBLE_DEVICES=0' before run the training command
"""

# -1 indicates using CPU for training; 
# 0 indicate we use the GPU with ID as 0, etc.
! export CUDA_VISIBLE_DEVICES=0 

# this is just an example, please refer to the readme for how to set hyperparameters
! python ../src/run_transformer_ner.py \
      --model_type bert \
      --pretrained_model bert-base-uncased \
      --data_dir ./ddi/ddi_bio/merge \
      --new_model_dir ./new_bert_ner_model \
      --overwrite_model_dir \
      --max_seq_length 128 \
      --data_has_offset_information \
      --save_model_core \
      --do_train \
      --model_selection_scoring strict-f_score-1 \
      --do_lower_case \
      --train_batch_size 8 \
      --train_steps 1000 \
      --learning_rate 1e-5 \
      --num_train_epochs 20 \
      --gradient_accumulation_steps 1 \
      --do_warmup \
      --seed 13 \
      --warmup_ratio 0.1 \
      --max_num_checkpoints 1 \
      --log_file ./log.txt \
      --progress_bar \
      --early_stop 3

## do prediction on each test set file and format prediction as brat output

In [ ]:
"""
running prediction

In our transformer package, we have the format conversion between bio and brat; bioc implemented

you still have to convert the txt files for prediction to BIO format,
but here you do not need to assign a real annotation label, we just use O as dummy
"""

# generate bio
test_root = Path(testing_root)
test_bio = "./ddi/ddi_bio/test"
output_root = Path(test_bio)
output_root.mkdir(parents=True, exist_ok=True)

for fn in test_root.glob("*.txt"):
    txt_fn = fn
    bio_fn = output_root / (fn.stem + ".bio.txt")
    
    txt, sents = pre_processing(txt_fn)
    annotations = [] # here we just use an empty list for annotation so that all words will be labeled as O
    nsents, sent_bound = generate_BIO(sents, annotations, file_id=fid, no_overlap=False)
    
    BIOdata_to_file(bio_fn, nsents)

In [ ]:
# run prediction
! export CUDA_VISIBLE_DEVICES=0

# use format 1 for BRAT, 2 for BioC, 0 as default for BIO
# see readme for more information on parameters
! python ../ClinicalTransformerNER/src/run_transformer_batch_prediction.py \
      --model_type bert \
      --pretrained_model ./new_bert_ner_model \
      --raw_text_dir $testing_root \ # this is the dir where original text file located
      --preprocessed_text_dir $test_bio \ # this is the dir where the BIO file located
      --output_dir ./ddi/results \
      --max_seq_length 128 \
      --do_lower_case \
      --eval_batch_size 8 \
      --log_file ./log.txt\
      --do_format 1 \
      --do_copy \
      --data_has_offset_information

# the bio prediction output will be generateed in ./ddi/results
# the brat prediction output will be generateed in ./ddi/results_formatted_output

## evaluation using brat eval script

In [ ]:
# run evaluation
# we have a brat_eval.py script used for evaluation of NER and relation extraction based on brat format

! python ../src/eval_scripts/brat_eval.py --f1 ./ddi/DDICorpusBrat/Test/MedLine --f2 ./ddi/results_formatted_output/